This notebook exists to experiment with different methods for classification

In [1]:
%run "Feature_Selection.ipynb" 

100%|████████████████████████████████████████████████████████████████████████████████| 202/202 [00:02<00:00, 93.28it/s]


In [2]:
removed_params = ['rr_std', 'RMSSD', 'pNN50', 'mean', 'lf', 'hf', 'skews', 'rr_mean', 'power_ratio', 'std'] #convert to a set?

selected_params = {}
for key in params.keys():
    if key not in removed_params:
        selected_params[key] = params[key]
print(selected_params.keys())

dict_keys(['kurtosis', 'shannon_en'])


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
health_state = np.array(allowed_patients.get_diagnoses())

encoded_health_state = [1 if label == 'Unhealthy' else -1 for label in health_state]

print(len(health_state))

202


In [36]:
def get_balanced_accuracy(y_test, y_pred):
    """
    balanced accuracy....
    """
    num_healthy_true = np.sum([x=='Healthy' for x in y_test])
    num_unhealthy_true = len(y_test) - num_healthy_true
    count_healthy_accurate = 0
    count_unhealthy_accurate = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Unhealthy':
            count_unhealthy_accurate +=1
        elif y_pred[i] == y_test[i] == 'Healthy':
            count_healthy_accurate +=1
    healthy_percentage = count_healthy_accurate/num_healthy_true
    unhealthy_percentage = count_unhealthy_accurate/num_unhealthy_true
    balanced_accuracy = (healthy_percentage + unhealthy_percentage) * 0.5
    return balanced_accuracy

def get_specificity(y_test, y_pred):
    """
    
    """
    true_negative = 0
    false_positive = 0
    for i in range(0, len(y_test)):
        if y_pred[i] == y_test[i] == 'Healthy':
            true_negative += 1
        elif y_pred[i] != y_test[i] and y_test[i] == 'Healthy':
            false_positive += 1
    
    return true_negative / (true_negative+false_positive)
            
    
    
    
def scoring_function(model, X, y):
    """
    change to incorporate balanced accuracy 
    """
    
    y_pred = model.predict(X)
    y_test = y
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    specificity = get_specificity(y_test, y_pred)
    
    
    return balanced_acc*0.8 + specificity*0.2

## SVM

In [29]:
def convert_dict_to_array(selected_params):
    no_features = len(selected_params)
    selected_params_array = np.zeros((no_patients, no_features))
    
    for i, values in enumerate(selected_params.values()):
        selected_params_array[:, i] = values
    return selected_params_array, no_features

        
selected_params_array, no_features = convert_dict_to_array(selected_params)


### Basic SVM

In [30]:
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

#init and train model, using radial basis functions
svm_model = SVC(kernel='rbf', gamma='scale', probability=True, class_weight = 'balanced')  #'scale' normalises data, prevents overfitting
#probability = True allows calculation of probabilities through 5-fold CV

svm_model.fit(X_train, y_train)

#predicting probabilities
probabilities = svm_model.predict_proba(X_test)
print(probabilities)


#predictions
y_pred = svm_model.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)
print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

print(np.sum([x=='Healthy' for x in y_test]))
print(y_test, y_pred)

#view a classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

[[0.13935485 0.86064515]
 [0.17318031 0.82681969]
 [0.17695851 0.82304149]
 [0.28021839 0.71978161]
 [0.15852536 0.84147464]
 [0.60847956 0.39152044]
 [0.17027135 0.82972865]
 [0.19431613 0.80568387]
 [0.18938976 0.81061024]
 [0.60037161 0.39962839]
 [0.60826553 0.39173447]
 [0.11181964 0.88818036]
 [0.10684561 0.89315439]
 [0.10961225 0.89038775]
 [0.24883956 0.75116044]
 [0.12920389 0.87079611]
 [0.10989632 0.89010368]
 [0.11203009 0.88796991]
 [0.57485109 0.42514891]
 [0.36472078 0.63527922]
 [0.10702777 0.89297223]
 [0.26351138 0.73648862]
 [0.10653087 0.89346913]
 [0.10780118 0.89219882]
 [0.35634019 0.64365981]
 [0.51644449 0.48355551]
 [0.13114897 0.86885103]
 [0.59506026 0.40493974]
 [0.52342868 0.47657132]
 [0.11231117 0.88768883]
 [0.2836013  0.7163987 ]
 [0.209102   0.790898  ]
 [0.10889412 0.89110588]
 [0.53823666 0.46176334]
 [0.42254209 0.57745791]
 [0.10658577 0.89341423]
 [0.11539474 0.88460526]
 [0.24788616 0.75211384]
 [0.11287429 0.88712571]
 [0.1127569  0.8872431 ]


### Hyper Parameter Tuning
- c is inversely proportional to l2 regularisation parameter
- gamma determines scaling of features


In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score

#recall needs encoded_health_state
#splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_params_array, health_state, test_size=0.3, stratify=health_state)

# define hyperparameter grid to test
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

#initialise
svc = SVC(class_weight='balanced')

# perform grid search
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring=scoring_function)
grid_search.fit(X_train, y_train)


print(grid_search.best_params_)

# train with best parameters
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, y_train)

{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}


SVC(C=0.1, class_weight='balanced')

In [38]:
#predictions
y_pred = best_svc.predict(X_test)

#evaluating accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = get_balanced_accuracy(y_test, y_pred)

print("Accuracy:", accuracy)
print('balanced accuracy:', balanced_accuracy)

Accuracy: 0.6557377049180327
balanced accuracy: 0.6261398176291793


### Stratified k-fold validation

need to weight the train and test sets to cope with difference with numbers of healthy and unhealthy, stratified k-fold validation

In [39]:
from sklearn.model_selection import StratifiedKFold

n_splits = 3 #3 fold validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True) #can do repeated skf for better validation


accuracies=[]
balanced_accuracies=[]
y_pred_list = []
y_test_list=[]
for train_index, test_index in skf.split(selected_params_array, health_state):
    #getting test and train data sets
    X_train, X_test = selected_params_array[train_index], selected_params_array[test_index]
    y_train, y_test = health_state[train_index], health_state[test_index]
    
    y_test_list.append(y_test)
    #training model with tuned hyperparameters
    tuned_svc = grid_search.best_estimator_
    tuned_svc.fit(X_train, y_train)
    
    #evaluating model
    y_pred = tuned_svc.predict(X_test)
    y_pred_list.append(y_pred)
    accuracies.append(accuracy_score(y_test, y_pred))
    balanced_accuracies.append((get_balanced_accuracy(y_test, y_pred)))

In [40]:
print(accuracies)
print(balanced_accuracies)

[0.7794117647058824, 0.5970149253731343, 0.6119402985074627]
[0.7692307692307692, 0.6692307692307693, 0.655128205128205]


In [41]:
print(np.mean(balanced_accuracies))

0.6978632478632477


In [20]:
print(y_pred_list[2], y_test_list[2])

['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Healthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Healthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Healthy' 'Unhealthy' 'Healthy' 'Healthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Healthy' 'Unhealthy' 'Healthy' 'Healthy' 'Unhealthy' 'Healthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Healthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Healthy' 'Unhealthy' 'Unhealthy'
 'Healthy' 'Healthy' 'Healthy' 'Healthy' 'Unhealthy' 'Unhealthy' 'Healthy'
 'Healthy' 'Healthy' 'Unhealthy' 'Healthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'] ['Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy' 'Unhealthy'
 'U

In [ ]:
print(y_pred, y_test)

In [35]:
print("Classification Report:")
print(classification_report(y_test_list[1], y_pred_list[1]))

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.00      0.00      0.00        15
   Unhealthy       0.78      1.00      0.87        52

    accuracy                           0.78        67
   macro avg       0.39      0.50      0.44        67
weighted avg       0.60      0.78      0.68        67



C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\court\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Comparing Accuracy
- MSE - average between true and predicted values
- R2 score - measure of proportion of variance in dependent variable (target) that is explained by independent variables (features), indicates how well the model captures the variations in the data

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer, f1_score, roc_auc_score



# Assuming selected_params_array and health_state are defined
X = selected_params_array
y = health_state

# Get the tuned model (best estimator from grid search)
tuned_svc = grid_search.best_estimator_

# Define accuracy as the scoring metric


# Perform cross-validation
n_splits = 5  # 5-fold validation
accuracies = cross_val_score(tuned_svc, X, y, cv=n_splits, scoring=roc_auc_scorer)

# Calculate the mean accuracy
mean_accuracy = accuracies.mean()
print(f'Mean Accuracy: {mean_accuracy}')


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

#calculate MSE
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

#calculate R2
r2_lasso = r2_score(y_test, y_pred_lasso)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("LASSO Regression:")
print("MSE:", mse_lasso)
print("R2 Score:", r2_lasso)
print()
print("Ridge Regression:")
print("MSE:", mse_ridge)
print("R2 Score:", r2_ridge)